In [3]:
## change to our functinon
from stage import getStage, fullPrint
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
import random
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
def evalOneMax(individual):
    print(individual[0])
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    tickerIndex = random.randint(0,len(yf.tickers_nasdaq())-1)
    for symbol in yf.tickers_nasdaq():
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        print(index, allocation, total, len(element.holding))
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    print(transactionCopy.iloc[-1]['total'])
    transactionCopy.to_csv("transaction/checkBug.csv")
    transactionCopy.to_pickle("transaction/checkBug.pkl")
    return transactionCopy.iloc[-1]['total']
evalOneMax([[0.0657188272423509, 2.498238289039671, 0.37709390631628703, 0.9045961833666701, 1.1888500273351494, 1.2760867811720553, 0.9051048571612113, 0.8519962174605497]])

[0.0657188272423509, 2.498238289039671, 0.37709390631628703, 0.9045961833666701, 1.1888500273351494, 1.2760867811720553, 0.9051048571612113, 0.8519962174605497]
100


100

In [4]:
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544,1.5]).to_csv("result.csv")
from stageOld import getFullDfOld
getFullDfOld("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544,1.5]).to_csv("resultOld.csv")

6484


ModuleNotFoundError: No module named 'stageOld'

In [ ]:
# from stage import getStage
# getStage("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544])
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544,1.5]).to_csv("result.csv")

2000-02-28 00:00:00 0.888392984867096
0.888392984867096
2000-03-20 00:00:00 1.1160709857940674
1.1160709857940674
